In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import emcee 
import pyimfit 
from astropy.io import fits
import corner
from IPython.display import Latex
import sys
import warnings
import tqdm


plt.rcParams['image.cmap'] = 'Blues'
warnings.filterwarnings("ignore")

imageFile = "../fit/agn.fits"
imageAGN = fits.getdata(imageFile)

epsf = fits.getdata("../psfConstruction/epsf2.fits")

In [2]:
def find_highest_indices(arr):
    flattened_arr = np.array(arr).flatten()
    max_indices = np.unravel_index(np.argsort(flattened_arr)[-2:], arr.shape)
    return max_indices


def galaxy_model(X0, Y0, X1, Y1, Xss0, Yss0, Xss1, Yss1, Xlim, Ylim,
                 PA_ss, ell_ss, n_ss, I_ss, r_ss, Itot,
                 PA_lim, ell_lim, I_lim, sigma_lim, Iss_lim, rss_lim):
    """create galaxy model for ML fit
        including: ps: 1 psf + 1 sersic
                   psps: 2 psf + 2 sersic"""

    sersic = {'PA': [PA_ss, PA_lim[0],PA_lim[1]], 'ell_bulge': [ell_ss, ell_lim[0],ell_lim[1]], 'n': [n_ss, 'fixed'],
    'I_e': [I_ss, Iss_lim[0],Iss_lim[1]], 'r_e': [r_ss, rss_lim[0],rss_lim[1]]}
    sersic_dict = {'name': "Sersic", 'label': "bulge", 'parameters': sersic}
    
    psf = {'I_tot' : Itot}
    psf_dict = {'name': "PointSource", 'label': "psf", 'parameters': psf}

    funcset_dict_psf0 = {'X0': [X0,Xlim[0],Xlim[1]], 'Y0': [Y0, Ylim[0],Ylim[1]], 
                    'function_list': [psf_dict]}
    funcset_dict_psf1 = {'X0': [X1,Xlim[0],Xlim[1]], 'Y0': [Y1, Ylim[0],Ylim[1]], 
                    'function_list': [psf_dict]}
    funcset_dict_sersic0 = {'X0': [Xss0,Xlim[0],Xlim[1]], 'Y0': [Yss0,Ylim[0],Ylim[1]], 
                   'function_list': [sersic_dict]}
    funcset_dict_sersic1 = {'X0': [Xss1,Xlim[0],Xlim[1]], 'Y0': [Yss1,Ylim[0],Ylim[1]], 
                   'function_list': [sersic_dict]}
    
    funcset_dict_psfser0 = {'X0': [X0,Xlim[0],Xlim[1]], 'Y0': [Y0, Ylim[0],Ylim[1]], 
                    'function_list': [psf_dict, sersic_dict]}
    funcset_dict_psfser1 = {'X0': [X1,Xlim[0],Xlim[1]], 'Y0': [Y1, Ylim[0],Ylim[1]], 
                    'function_list': [psf_dict, sersic_dict]}

    
    model_dict_ps_1c = {'function_sets': [funcset_dict_psfser0]}
    model_dict_psps_1c = {'function_sets': [funcset_dict_psfser0,funcset_dict_psfser1]}
    
    model_dict_ps_2c = {'function_sets': [funcset_dict_psf0,funcset_dict_sersic0]}
    model_dict_psps_2c = {'function_sets': [funcset_dict_psf0, funcset_dict_psf1,funcset_dict_sersic0,funcset_dict_sersic1]}
    
    model_ps1c = pyimfit.ModelDescription.dict_to_ModelDescription(model_dict_ps_1c)
    model_psps1c = pyimfit.ModelDescription.dict_to_ModelDescription(model_dict_psps_1c)
    model_ps2c = pyimfit.ModelDescription.dict_to_ModelDescription(model_dict_ps_2c)
    model_psps2c = pyimfit.ModelDescription.dict_to_ModelDescription(model_dict_psps_2c)
    return model_ps1c, model_psps1c, model_ps2c, model_psps2c



def doFit(models,epsf): 
    """do fit with data using PSF oversampled by factor of 4 over whole image
       return fitters"""
    fitters = []
    # oversamped PSF
    psfOsamp = pyimfit.MakePsfOversampler(epsf, 4, (0,100,0,100))
    osampleList = [psfOsamp]
    for i in tqdm.tqdm(range(len(models)), desc="Fitting Models"):
        imfit_fitter = pyimfit.Imfit(models[i],psf=epsf)
        imfit_fitter.loadData(imageAGN, psf_oversampling_list=osampleList, gain=9.942e-1, read_noise=0.22, original_sky=15.683)
        imfit_fitter.doFit()
        fitters.append(imfit_fitter)
    return fitters

In [3]:
starpos = find_highest_indices(imageAGN)
center = (starpos[0]+starpos[1])/2
Imax = imageAGN.max()
itot=130

models_n1 = galaxy_model(X0=starpos[0][0], Y0=starpos[0][1], X1=starpos[1][0], Y1=starpos[1][1], 
                                                 Xss0=starpos[0][0], Yss0=starpos[0][1], Xss1=starpos[1][0], Yss1=starpos[1][1],
                                                 Xlim=[0,100], Ylim=[0,100],
                                                 PA_ss=200, ell_ss=0.1, n_ss=1, I_ss=1, r_ss=20, Itot=itot,
                                                 PA_lim=[0,360], ell_lim=[0.0,1.0], I_lim=[0,Imax], sigma_lim=[0,15],
                                      Iss_lim=[0,Imax], rss_lim=[0,100])

In [10]:
def doFit(models,epsf): 
    """do fit with data using PSF oversampled by factor of 4 over whole image
       return fitters"""
    fitters = []
    # oversamped PSF
    psfOsamp = pyimfit.MakePsfOversampler(epsf, 4, (0,100,0,100))
    osampleList = [psfOsamp]
    for i in tqdm.tqdm(range(len(models)), desc="Fitting Models"):
        imfit_fitter = pyimfit.Imfit(models[i],psf=epsf)
        imfit_fitter.loadData(imageAGN, psf_oversampling_list=osampleList, gain=9.942e-1, read_noise=0.22, original_sky=15.683)
        imfit_fitter.doFit()
        fitters.append(imfit_fitter)
    return fitters

fitters = doFit(models_n1, epsf)

PsfOversampling: starting initialization...
141
PsfOversampling: done.


Fitting Models:  75%|███████████████████████████████████████████████████▊                 | 3/4 [02:43<00:55, 55.24s/it]

PsfOversampling: starting __dealloc__.
PsfOversampling: done with __dealloc__.


Fitting Models: 100%|█████████████████████████████████████████████████████████████████████| 4/4 [02:51<00:00, 42.91s/it]

PsfOversampling: starting __dealloc__.
PsfOversampling: done with __dealloc__.


In [11]:
fitters[1].getFitResult()

            aic: 919.5264382047388
            bic: 1020.4291403177634
   fitConverged: True
        fitStat: 891.4843751100968
 fitStatReduced: 0.08927342029942889
          nIter: 40
      paramErrs: array([8.20637084e-02, 8.04210389e-02, 1.70088560e+03, 2.90781957e+08,
       0.00000000e+00, 0.00000000e+00, 2.74135666e-01, 1.82570147e+00,
       7.98688519e-02, 7.06316378e-02, 2.15870135e+03, 1.90097936e+00,
       2.17358427e-02, 0.00000000e+00, 9.27574113e-01, 4.17207166e-01])
         params: array([5.34005664e+01, 5.34890951e+01, 4.32690719e+04, 3.54477253e+02,
       0.00000000e+00, 1.00000000e+00, 1.40480858e+00, 2.08115696e+01,
       4.83959561e+01, 5.76541455e+01, 3.80936814e+04, 7.43091213e+01,
       3.85449711e-01, 1.00000000e+00, 1.54900204e+01, 1.08697724e+01])
     solverName: 'LM'

In [9]:
fitter[0].getFitResult()

            aic: 899.5659018685352
            bic: 1000.4686039815598
   fitConverged: True
        fitStat: 871.5238387738932
 fitStatReduced: 0.08727456827297148
          nIter: 34
      paramErrs: array([8.47248824e-02, 8.29446984e-02, 8.83745163e+01, 6.29677594e+00,
       7.72343189e-02, 0.00000000e+00, 2.73551128e-01, 3.32180119e+00,
       7.90066094e-02, 7.31829516e-02, 1.12405677e+02, 2.13457292e+00,
       2.27458912e-02, 0.00000000e+00, 9.43961872e-01, 4.79783933e-01])
         params: array([5.34139408e+01, 5.35234688e+01, 2.17767031e+03, 2.95123627e+02,
       4.09985547e-01, 1.00000000e+00, 1.36430495e+00, 2.77666428e+01,
       4.83992738e+01, 5.76743064e+01, 1.93033344e+03, 2.48553926e+02,
       3.82793236e-01, 1.00000000e+00, 1.58382706e+01, 1.08054954e+01])
     solverName: 'LM'

In [16]:
np. set_printoptions(suppress=True    )
np.ceil(np.array([5.34139408e+01, 5.35234688e+01, 2.17767031e+03, 2.95123627e+02,
       4.09985547e-01, 1.00000000e+00, 1.36430495e+00, 2.77666428e+01,
       4.83992738e+01, 5.76743064e+01, 1.93033344e+03, 2.48553926e+02,
       3.82793236e-01, 1.00000000e+00, 1.58382706e+01, 1.08054954e+01]))

array([  54.,   54., 2178.,  296.,    1.,    1.,    2.,   28.,   49.,
         58., 1931.,  249.,    1.,    1.,   16.,   11.])

In [17]:
np.ceil(np.array([5.34005664e+01, 5.34890951e+01, 4.32690719e+04, 3.54477253e+02,
       0.00000000e+00, 1.00000000e+00, 1.40480858e+00, 2.08115696e+01,
       4.83959561e+01, 5.76541455e+01, 3.80936814e+04, 7.43091213e+01,
       3.85449711e-01, 1.00000000e+00, 1.54900204e+01, 1.08697724e+01]))

array([   54.,    54., 43270.,   355.,     0.,     1.,     2.,    21.,
          49.,    58., 38094.,    75.,     1.,     1.,    16.,    11.])

In [15]:
psfOsamp = pyimfit.MakePsfOversampler(epsf, 4, (0,100,0,100))
osampleList = [psfOsamp]

fitter = pyimfit.Imfit(models_n1[0],psf=epsf)
fitter.loadData(imageAGN, psf_oversampling_list=osampleList, gain=9.942e-1, read_noise=0.22, original_sky=15.683)
fitter.doFit()
print(fitter.getFitResult())

fitter = pyimfit.Imfit(models_n1[1],psf=epsf)
fitter.loadData(imageAGN, psf_oversampling_list=osampleList, gain=9.942e-1, read_noise=0.22, original_sky=15.683)
fitter.doFit()
print(fitter.getFitResult())

PsfOversampling: starting initialization...
141
PsfOversampling: done.
PsfOversampling: starting __dealloc__.
PsfOversampling: done with __dealloc__.
            aic: 1653.1479253308087
            bic: 1703.609098967468
   fitConverged: True
        fitStat: 1639.1367163636348
 fitStatReduced: 0.1640284915804698
          nIter: 36
      paramErrs: array([7.18010654e-02, 6.08843537e-02, 1.02677539e+02, 1.22878200e+00,
       1.31169996e-02, 0.00000000e+00, 7.33820293e-01, 2.08747383e-01])
         params: array([4.94873561e+01, 5.67360988e+01, 1.37318644e+03, 6.59741105e+01,
       3.77094930e-01, 1.00000000e+00, 2.29771397e+01, 1.09635638e+01])
     solverName: 'LM'
            aic: 919.5264382047388
            bic: 1020.4291403177634
   fitConverged: True
        fitStat: 891.4843751100968
 fitStatReduced: 0.08927342029942889
          nIter: 40
      paramErrs: array([8.20637084e-02, 8.04210389e-02, 1.70088560e+03, 2.90781957e+08,
       0.00000000e+00, 0.00000000e+00, 2.74135666e

In [18]:
dir(fitter)

['AIC',
 'BIC',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_chunkSize',
 '_dataSet',
 '_debugLevel',
 '_finalSetupDone',
 '_fitDone',
 '_fitStatComputed',
 '_lastSolverUsed',
 '_mask',
 '_maxThreads',
 '_modelDescr',
 '_modelObjectWrapper',
 '_normalizePSF',
 '_psf',
 '_setupModel',
 '_subsampling',
 '_updateModelDescription',
 '_verboseLevel',
 '_zeroPoint',
 'computeFitStatistic',
 'doFit',
 'fit',
 'fitConverged',
 'fitError',
 'fitStatistic',
 'fitTerminated',
 'getFitResult',
 'getModelAsDict',
 'getModelDescription',
 'getModelFluxes',
 'getModelImage',
 'getModelMagnitudes',
 'getParameterErrors',
 'getParameterLimits',
 'getRa

In [20]:
fitter.zeroPoint?